# CDMS Chapter 4 <a id='top' class="tocSkip"> </a>


The CDAT software was developed by LLNL. This work was performed under the auspices of the U.S. Department of Energy by Lawrence Livermore National Laboratory under Contract DE-AC52-07NA27344.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Regridding-Data" data-toc-modified-id="Regridding-Data-1">Regridding Data</a></span></li><li><span><a href="#Overview" data-toc-modified-id="Overview-2">Overview</a></span><ul class="toc-item"><li><span><a href="#CDMS-Horizontal-Regridder" data-toc-modified-id="CDMS-Horizontal-Regridder-2.1">CDMS Horizontal Regridder</a></span><ul class="toc-item"><li><span><a href="#First-Regridding-Example:" data-toc-modified-id="First-Regridding-Example:-2.1.1">First Regridding Example:</a></span></li></ul></li><li><span><a href="#Regridder-function" data-toc-modified-id="Regridder-function-2.2">Regridder function</a></span><ul class="toc-item"><li><span><a href="#Efficient-method-using-first-example" data-toc-modified-id="Efficient-method-using-first-example-2.2.1">Efficient method using first example</a></span></li></ul></li><li><span><a href="#ESMF-Horizontal-Regridder" data-toc-modified-id="ESMF-Horizontal-Regridder-2.3">ESMF Horizontal Regridder</a></span></li><li><span><a href="#Regridding-Data" data-toc-modified-id="Regridding-Data-2.4">Regridding Data</a></span></li><li><span><a href="#Pressure-Level-Regridder" data-toc-modified-id="Pressure-Level-Regridder-2.5">Pressure-Level Regridder</a></span></li><li><span><a href="#Cross-Section-Regridder" data-toc-modified-id="Cross-Section-Regridder-2.6">Cross-Section Regridder</a></span></li><li><span><a href="#Regrid-Module" data-toc-modified-id="Regrid-Module-2.7">Regrid Module</a></span></li><li><span><a href="#ESMF-Regridder" data-toc-modified-id="ESMF-Regridder-2.8">ESMF Regridder</a></span></li><li><span><a href="#ESMF-Regridder-Constructor" data-toc-modified-id="ESMF-Regridder-Constructor-2.9">ESMF Regridder Constructor</a></span></li><li><span><a href="#ESMF-Regridder-Functions" data-toc-modified-id="ESMF-Regridder-Functions-2.10">ESMF Regridder Functions</a></span><ul class="toc-item"><li><span><a href="#Example-1" data-toc-modified-id="Example-1-2.10.1">Example 1</a></span></li><li><span><a href="#Example-2" data-toc-modified-id="Example-2-2.10.2">Example 2</a></span></li></ul></li><li><span><a href="#Zonal-Mean-Regridder" data-toc-modified-id="Zonal-Mean-Regridder-2.11">Zonal Mean Regridder</a></span></li><li><span><a href="#SCRIP-Regridder" data-toc-modified-id="SCRIP-Regridder-2.12">SCRIP Regridder</a></span></li></ul></li></ul></div>

# Regridding Data



# Overview

[Back to Top](#top)

CDMS provides several methods for interpolating gridded data:

-  From one rectangular, lat-lon grid to another (CDMS regridder)
-  Between any two lat-lon grids (ESMF regridder)
-  From one set of pressure levels to another
-  From one vertical (lat/level) cross-section to another vertical
   cross-section.


## CDMS Horizontal Regridder

[Back to Top](#top)

The simplest method to regrid a variable from one rectangular, lat/lon
grid to another is to use the regrid function defined for variables.
This function takes the target grid as an argument, and returns the
variable regridded to the target grid:


### First Regridding Example:

[Back to Top](#top)

Open input files.  

Surface Pressure will be regridded.

In [2]:
import cdms2
f1 = cdms2.open("/global/cscratch1/sd/cmip6/CMIP6/CMIP/MIROC/MIROC6/historical/r1i1p1f1/Amon/ps/gn/v20181212/ps_Amon_MIROC6_historical_r1i1p1f1_gn_195001-201412.nc")
f2 = cdms2.open("/global/cscratch1/sd/cmip6/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/historical/r1i1p1f1/Amon/ps/gr/v20180803/ps_Amon_IPSL-CM6A-LR_historical_r1i1p1f1_gr_185001-201412.nc")

Read in the data

In [3]:
ps_miroc=f1('ps', time=("2000", "2010", "con"))  
print("input grid:",ps_miroc.shape)

input grid: (120, 128, 256)


Read needed information from the target file.

* Get the file variable (no data read with square brackets)
* Get the target grid

In [4]:
ps_ipsl=f2['ps']  
outgrid = ps_ipsl.getGrid() 
print("desired grid:",outgrid.shape)
print("regridding input data...")

desired grid: (143, 144)
regridding input data...


Regrid "miroc" to fit "ipsl" grid.

In [5]:
new = ps_miroc.regrid(outgrid, method="linear")
print("new regridded input data:",new.shape)

/opt/conda/lib/python3.7/site-packages/cdms2/avariable.py:1313: Warning: 
avariable.regrid: We chose regridTool = esmf for you among the following choices:
   Tools ->    'regrid2' (old behavior)
               'esmf' (conserve, patch, linear) or
               'libcf' (linear)
  warnings.warn(message, Warning)


new regridded input data: (120, 143, 144)


You can select different regrid methods and regrid tools.

Three regrid tools are available.

* libcf (UNIDATA linear)
* esmf
* regrid (LLNL regridder)

Depending on the regrid tool you select only some regrid methods are allowed.

* linear all tools
* bilinear all toosl
* conservative (ESMF only)
* patch (ESMF Only)


In [6]:
new = ps_miroc.regrid(outgrid, regridTool="regrid2", method="bilinear")
print("new regridded input data:",new.shape)

new regridded input data: (120, 143, 144)


In [7]:
new = ps_miroc.regrid(outgrid, regridTool="esmf", method="patch")
print("new regridded input data:",new.shape)

new regridded input data: (120, 143, 144)


## Regridder function

[Back to Top](#top)

A somewhat more efficient method is to create a regridder function. This
has the advantage that the mapping is created ``only once`` and can be used for **multiple arrays**. Also, this method can be used with data in the form of an ``MV2.MaskedArray``. 

The steps in this process are:

* Given an input grid and output grid, generate a regridder function.
* Call the regridder function on a Numpy array, resulting in an array
   defined on the output grid. 
* The regridder function can be called with any array or variable defined on the input grid.

### Efficient method using first example

[Back to Top](#top)

The following example illustrates this process. 

1. Makes the Regridder class available from the regrid module.

In [8]:
import cdms2
from regrid2 import Regridder

2. Opens the input dataset.
2. Gets the variable object named ‘clt’. No data is read using square brackets ``[]``.

In [9]:
ps_miroc = f1['ps']
ingrid = ps_miroc.getGrid() # Get the input grid

4. Opens a dataset to retrieve the output grid.

2. The output grid is the grid associated with the variable named **ozone** in dataset ``g``. 

**Note**: Just the grid is retrieved, not the data using square brackets [].

In [10]:
ps_ipsl = f2['ps'].getGrid()  # Get the output grid

6. Generates a regridder function regridfunc.

In [11]:
regridfunc = Regridder(ingrid, outgrid)  # Create the "Regridder function"

/opt/conda/lib/python3.7/site-packages/regrid2/horizontal.py:376: Warning: While this will work for now, please note that the Regridder class has been renamed Horizontal, the name 'Regridder' will be deprecated in future version. Please edit your code accordingly
  Warning)


7. Calls the regridder
function on that data, all data for variable cltf will be read and execute the regridder function on that data, resulting in a transient variable cltnew.

In [12]:
new = regridfunc(ps_miroc(time=("2000", "2010", "con")))
print(new.shape)

(120, 143, 144)


8. Close files.

In [13]:
f1.close()
f2.close()

## ESMF Horizontal Regridder

[Back to Top](#top)

To interpolate between grids where one or both grids is non-rectangular,
CDMS provides an interface to the ESMF regridder package. (https://www.earthsystemcog.org/projects/esmf/). 


## Regridding Data

[Back to Top](#top)



Create the regridder using remapper file.

In [14]:
import regrid2, cdms2
# Read the regridder from the remapper file
remapf = cdms2.open('rmp_T42_to_POP43_conserv.nc')
regridf = regrid2.readRegridder(remapf)
remapf.close()

Then read the input data and regrid

In [15]:
# Get the source variable
f = cdms2.open('xieArkin-T42.nc')
t42prc = f('prc')
f.close()
# Regrid the source variable
popdat = regridf(t42prc)
print("input grid:", t42prc.shape)
print("output grid:",popdat.shape)

input grid: (216, 64, 128)
output grid: (216, 128, 192)


**Note:** That ``t42dat`` can have rank greater than 2. The trailing
dimensions must match the input grid shape. 

For example, if ``t42dat``has shape (216, 64, 128), then the input grid must have shape (64, 128). Similarly if the variable had a generic grid with shape (128, 192) the
last dimension of the variable would have length (128, 192).

## Pressure-Level Regridder

[Back to Top](#top)

To regrid a variable which is a function of latitude, longitude,
pressure level, and (optionally) time to a new set of pressure levels,
use the ``pressureRegrid`` function defined for variables. This function
takes an axis representing the target set of pressure levels, and
returns a new variable ``d`` regridded to that dimension.

In [16]:
f=cdms2.open("/global/cscratch1/sd/cmip6/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/historical/r1i1p1f1/Amon/hus/gr/v20180803/hus_Amon_IPSL-CM6A-LR_historical_r1i1p1f1_gr_185001-201412.nc")
hus = f("hus", time=slice(0,12))  # One year worth
print(hus.shape)
print(hus.getAxisIds())
print(hus.getLevel())
result = hus.pressureRegrid(cdms2.createAxis([101300.0]))
print(result.shape)


(12, 19, 143, 144)
['time', 'plev', 'lat', 'lon']
   id: plev
   Designated a level axis.
   units:  Pa
   Length: 19
   First:  100000.0
   Last:   100.0
   Other axis attributes:
      axis: Z
      standard_name: air_pressure
      long_name: pressure
      positive: down
      realtopology: linear
   Python id:  0x2aaad093d630

(12, 1, 143, 144)


## Cross-Section Regridder

[Back to Top](#top)

To regrid a variable which is a function of latitude, height, and
(optionally) time to a new latitude/height cross-section, use the
``crossSectionRegridder`` defined for variables. This function takes as
arguments the new latitudes and heights, and returns the variable
regridded to those axes.

In [17]:
f=cdms2.open("/global/cscratch1/sd/cmip6/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/historical/r1i1p1f1/Amon/hus/gr/v20180803/hus_Amon_IPSL-CM6A-LR_historical_r1i1p1f1_gr_185001-201412.nc")
hus = f("hus", time=slice(0,12))  # One year worth
print(hus.shape)

levOut=cdms2.createAxis([100000.0,95000.])
levOut.designateLevel()

latOut=cdms2.createAxis(hus.getLatitude()[10:20])
latOut.designateLatitude()

print(hus.getAxisIds())

# regrid to new latitude/height 
# We cannot have time in it so only regird 1st time step
out = hus[0].crossSectionRegrid(levOut, latOut)
print(out.shape)

(12, 19, 143, 144)
['time', 'plev', 'lat', 'lon']
(2, 10, 144)


## Regrid Module


[Back to Top](#top)

The ``regrid`` module implements the CDMS regridding functionality as
well as the ESMF interface. Although this module is not strictly a part
of CDMS, it is designed to work with CDMS objects.

## ESMF Regridder

[Back to Top](#top)

ESMF regridder functions are created with the ``regrid.readRegridder``
function:

## ESMF Regridder Constructor

[Back to Top](#top)

 "``regridFunction = regrid.readRegridder(fileobj, mapMethod=None, checkGrid=1)``", "Read a regridder from an open CDMS file object.
**fileobj** is a CDMS file object, as returned from **cdms.open**.

**mapMethod** is one of:

 * **'conservative'**: conservative remapper, suitable where area-integrated fields such as water or heat fluxes must be conserved.
 * **'bilinear'**: bilinear interpolation
 * **'bicubic'**: bicubic interpolation
 
**Note:** It is only necessary to specify the map method if it is not defined in the file.
 
 
* If **checkGrid** is 1 (default), the grid cells are checked for convexity, and 'repaired' if necessary. Grid cells may appear to be nonconvex if they cross a ``0 / 2pi`` boundary. The repair consists of shifting the cell vertices to the same side modulo 360 degrees."


## ESMF Regridder Functions


[Back to Top](#top)

An ESMF regridder function is an instance of the Regridder class.

It only work for ``SCRIP`` netcdf files at this time.

Such a function is created by calling the regrid.readRegridder method.
Typical usage is straightforward:

In [17]:
import cdms2
import regrid2
remapf = cdms2.open('rmp_T42_to_POP43_conserv.nc')
regridf = regrid2.readRegridder(remapf)
f = cdms2.open('xieArkin-T42.nc')
t42prc = f('prc')
f.close()
# Regrid the source variable
popdat = regridf(t42prc)
print(t42prc.shape)
print(popdat.shape)

(216, 64, 128)
(216, 128, 192)


The bicubic regridder takes four arguments:

### Example 1

[Back to Top](#top)

Regrid data to a uniform output grid.

In [18]:
import cdms2
from regrid2 import Regridder
f = cdms2.open("/global/cscratch1/sd/cmip6/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/historical/r1i1p1f1/Amon/ps/gr/v20180803/ps_Amon_IPSL-CM6A-LR_historical_r1i1p1f1_gr_185001-201412.nc")
ps = f('ps', time=slice(0,120))  # 10 years
ingrid = ps.getGrid()

# Create a 181 x 361 degree output grid. Note that this grid is not associated with a file or dataset."
# USING: 
# CreateUniformGrid(startLat, nlat, deltaLat, startLon, nlon, deltaLon, order='yx', mask=None)
#
outgrid2 = cdms2.createUniformGrid(90.0, 181, -1.0, 0.0, 361 , 1.0)

# Create the regridder function.
regridFunc = Regridder(ingrid, outgrid2)
# Read all data and regrid. The missing data value is obtained from variable rlsf"
new = regridFunc(ps)
print("old grid for Cloud Top Variable:", ps.shape)
print("new grid for Cloud Tope Variable:", new.shape)

f.close()

old grid for Cloud Top Variable: (120, 143, 144)
new grid for Cloud Tope Variable: (120, 181, 361)


### Example 2

[Back to Top](#top)

Return the area fraction of the source (input) grid cell that
participates in the regridding. The array is 1-D, with length equal to
the number of cells in the input grid.

Get a mask from a separate file, and set as the input grid mask.

In [19]:
import cdms2
from regrid2 import Regridder
#
f = cdms2.open("/global/cscratch1/sd/cmip6/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/historical/r1i1p1f1/Amon/ps/gr/v20180803/ps_Amon_IPSL-CM6A-LR_historical_r1i1p1f1_gr_185001-201412.nc")
ipsl = f['ps']
outgrid = ipsl.getGrid()

g = cdms2.open("/global/cscratch1/sd/cmip6/CMIP6/CMIP/MIROC/MIROC6/historical/r1i1p1f1/Amon/ps/gn/v20181212/ps_Amon_MIROC6_historical_r1i1p1f1_gn_195001-201412.nc")
miroc = g('ps', time=slice(0,120)) # 10 years
ingrid = miroc.getGrid()

regridFunc = Regridder(ingrid,outgrid)

print("Max:", miroc.max())
mask = miroc[:]<85000.
outArray = regridFunc(miroc, mask=mask)
print("Input mask shape:", mask.shape)
print("Output Mask shape:", outArray.mask.shape)
print("Input valid values:", miroc.count())
print("Output valid values vs target count:", outArray.count(), ipsl(time=slice(0,120)).count())
f.close()
g.close()

Max: 103997.23
Input mask shape: (120, 128, 256)
Output Mask shape: (120, 143, 144)
Input valid values: 3932160
Output valid values vs target count: 2247578 2471040


## Zonal Mean Regridder

[Back to Top](#top)

In [20]:
f = cdms2.open("/global/cscratch1/sd/cmip6/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/historical/r1i1p1f1/Amon/ps/gr/v20180803/ps_Amon_IPSL-CM6A-LR_historical_r1i1p1f1_gr_185001-201412.nc")
ipsl = f("ps", time=slice(0,120))
ingrid = ipsl.getGrid()
outgrid = cdms2.createZonalGrid(ingrid)
regridFunc = Regridder(ingrid,outgrid)
mean = regridFunc(ipsl)
print(ipsl.shape)
print(mean.shape)
f.close()

(120, 143, 144)
(120, 143, 1)


In [21]:
import cdms2
from cdms2.MV2 import *
from regrid2 import Regridder

f=cdms2.open("/global/cscratch1/sd/cmip6/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/historical/r1i1p1f1/Amon/hus/gr/v20180803/hus_Amon_IPSL-CM6A-LR_historical_r1i1p1f1_gr_185001-201412.nc")
var = f("hus", time=slice(0,12))  # One year worth

outgrid = cdms2.createUniformGrid(90.0, 46, -4.0, 0.0, 72, 5.0)

outlatw, outlonw = outgrid.getWeights()
outweights = outerproduct(outlatw, outlonw)

grid = var.getGrid()

sample = var[0,0]

latw, lonw = grid.getWeights()
weights = outerproduct(latw, lonw)

inmask = where(greater(absolute(sample),1.e15),0,1)

mean = add.reduce(ravel(inmask*weights*sample))/add.reduce(ravel(inmask*weights))

regridFunc = Regridder(grid, outgrid)
outsample, outmask = regridFunc(sample, mask=inmask, returnTuple=1)
outmean = add.reduce(ravel(outmask*outweights*outsample)) / add.reduce(ravel(outmask*outweights))

## SCRIP Regridder

[Back to Top](#top)

Regrid from a curvilinear to a generic grid, using a conservative
remapping. Compute the area-weighted means on input and output for
comparison.

In [22]:
import cdms2, regrid2, MV2

# Open the SCRIP remapping file and data file
fremap = cdms2.open('rmp_T42_to_C02562_conserv.nc')
fdat = cdms2.open('xieArkin-T42.nc')

# Input data array
dat = fdat('prc')[0,:]

# Read the ESMF regridder
regridf = regrid2.readRegridder(fremap)

# Regrid the variable
outdat = regridf(dat)

# Get the cell area and fraction arrays. Areas are computed only
# for conservative regridding.
srcfrac = regridf.getSourceFraction()
srcarea = regridf.getSourceArea()
dstfrac = regridf.getDestinationFraction()
dstarea = regridf.getDestinationArea()

# calculate area-weighted means
inmean = MV2.sum(srcfrac*srcarea*MV2.ravel(dat)) / MV2.sum(srcfrac*srcarea)
outmean = MV2.sum(dstfrac*dstarea*MV2.ravel(outdat)) / MV2.sum(dstfrac*dstarea)
print('Input mean:', inmean)
print('Output mean:', outmean)

fremap.close()
fdat.close()

Input mean: 2.6037650233869063
Output mean: 2.6037650233869014
